![](pics/header.png)

# MSP

## References

- stackoverflow: [cleanflight msp](https://stackoverflow.com/questions/42877001/how-do-i-read-gyro-information-from-cleanflight-using-msp)
- web.archive: [msp message](https://web.archive.org/web/20190715222846/http://www.stefanocottafavi.com/msp-the-multiwii-serial-protocol/)
- web.archive: [message value formats](https://web.archive.org/web/20160820205716/http://www.multiwii.com/wiki/index.php?title=Multiwii_Serial_Protocol)


# MSP

- Commands: message sent to controller with no response
- Request: message sent to controller with an expected response back
- Response: message returned from controller due to a request

The message structure is shown below:

![](pics/bits.png)

```
[Preamble][Direction][Length][Command][Payload][Checksum]
```

- Preamble: `$M`
- Direction:
    - `<` for message to controller (command or request)
    - `>` for message from controller (response)
- N: the number of bytes in the payload, N is 0 for requests.
- T: the type of message, values of 1xx identify requests while values of 2xx identify commands
- C: The checksum is computed as the XOR of size, type and payload bytes. The checksum of a request (no payload) equals the type.

```c
MSP_IDENT                =100,
MSP_STATUS               =101,
MSP_RAW_IMU              =102,
MSP_SERVO                =103,
MSP_MOTOR                =104,
MSP_RC                   =105,
MSP_RAW_GPS              =106,
MSP_COMP_GPS             =107,
MSP_ATTITUDE             =108,
MSP_ALTITUDE             =109,
MSP_ANALOG               =110,
MSP_RC_TUNING            =111,
MSP_PID                  =112,
MSP_BOX                  =113,
MSP_MISC                 =114,
MSP_MOTOR_PINS           =115,
MSP_BOXNAMES             =116,
MSP_PIDNAMES             =117,
MSP_SERVO_CONF           =120,

MSP_SET_RAW_RC           =200,
MSP_SET_RAW_GPS          =201,
MSP_SET_PID              =202,
MSP_SET_BOX              =203,
MSP_SET_RC_TUNING        =204,
MSP_ACC_CALIBRATION      =205,
MSP_MAG_CALIBRATION      =206,
MSP_SET_MISC             =207,
MSP_RESET_CONF           =208,
MSP_SELECT_SETTING       =210,
MSP_SET_HEAD             =211, // Not used
MSP_SET_SERVO_CONF       =212,
MSP_SET_MOTOR            =214,

MSP_BIND                 =241,

MSP_EEPROM_WRITE         =250,

MSP_DEBUGMSG             =253,
MSP_DEBUG                =254
```


In [1]:
import serial
import struct
from collections import namedtuple

In [2]:
# import serial
# import struct
# from collections import namedtuple

IDENT = namedtuple("IDENT", "version multitype msp_version capability")
IMU_RAW = namedtuple("IMU_RAW", "ax ay az gx gy gz mx my mz")
ATTITUDE = namedtuple("ATTITUDE", "roll pitch heading")
ALTITUDE = namedtuple("ALTITUDE", "alt vario")

class MSP:
    """
    """

    # 1xx identify requests
    MSP_IDENT = 100
    MSP_STATUS = 101
    MSP_RAW_IMU = 102
    MSP_ATTITUDE = 108
    MSP_ALTITUDE = 109

    # 2xx identify commands
    MSP_SET_RAW_RC = 200
    MSP_SET_MOTOR = 214

    def request(self, cmd):
        # msg = bytes(6)
        msg = struct.pack("<cccBBB",b"$",b"M",b"<",0,cmd,cmd)
        # c = self.checksum(msg[3:])
        # print(f">> c: {c}")
        # msg[5] = struct.pack("<B", c)
        return msg

    def response(self, msg):
        i = msg.find(b"$M>")+3
        len = int(msg[i])
        kind = int(msg[i+1])
        payload = msg[i+2:i+2+len]
        csum = msg[i+2+len]

        calcsum = self.checksum(msg[i:i+2+len])
        if csum != calcsum:
            print(f"{csum} != {calcsum}")
            return None
        # print(f"{csum} == {self.checksum(msg[i:i+2+len])}")
        # print(kind, csum, payload)
        ret = None
        if kind == self.MSP_IDENT:
            data = struct.unpack("<BBBI", payload)
            ret = IDENT(*data)
        elif kind == self.MSP_RAW_IMU:
            data = struct.unpack("<hhhhhhhhh", payload)
            ret = IMU_RAW(*data)
        elif kind == self.MSP_ATTITUDE:
            r,p,h = struct.unpack("<hhh", payload)
            ret = ATTITUDE(r/10.0, p/10.0, h)
        elif kind == self.MSP_ALTITUDE:
            data = struct.unpack("<ih", payload)
            ret = ALTITUDE(*data)

        return ret

    def checksum(self, msg):
        cs = 0
        for m in msg:
            cs ^= m
        return cs



# port = "/dev/tty.usbserial-013950B1"
# s = serial.Serial()
# s.baudrate = 115200
# s.timeout = 0.1
# s.port = port
# s.open()

# msp = MSP()
# msg = msp.request(MSP.MSP_IDENT)
# print(type(msg), msg)
# s.write(msg)
# resp = s.read(128)
# print(resp)
# rmsg = msp.response(resp)
# print(rmsg)


In [3]:
msp = MSP()
msg = msp.request(MSP.MSP_IDENT)
print(type(msg), msg)

<class 'bytes'> b'$M<\x00dd'
